In [ ]:
import mlflow
import mlflow.tracking._model_registry.utils
from dotenv import load_dotenv

load_dotenv()

# The login and mlflow model registery are set this way because of local development. If you are running this in Databricks, you can remove the login and use the regualr mlflow registry.

mlflow.login()

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

mlflow.set_experiment("/ConversationalRAGComplexLangGraph")

In [ ]:
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        streamable=True,
        pip_requirements="../../requirements.txt",
        python_model="ConversationalRAGComplexLangGraph.py",
        input_example={
            "messages": [
                {
                    "role": "user",
                    "content": "How did ETL work in the first generation platforms?",
                }
            ]
        },
    )

my_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [ ]:
my_model.predict(
    {
        "messages": [
            {"role": "user", "content": "What is databricks, use your search tool"}
        ]
    }
)

In [ ]:
for chunk in my_model.predict_stream(
    {
        "messages": [
            {"role": "user", "content": "What is databricks, use your search tool"}
        ]
    }
):
    print(chunk)